<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/IgboBERT%202.0/Fine_tuned_IgboBert_2_0_Merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Classification (PyTorch)-Fine-Tunning 

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [36]:
from datasets import concatenate_datasets, load_dataset, DatasetDict
ibo1 = load_dataset('masakhaner', 'ibo')
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo')



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
merged_dataset = []
for split in ['train','test', 'validation']:
  ibo1_split = load_dataset('masakhaner', 'ibo', split=split) 
  ibo2_split = load_dataset('masakhane/masakhaner2', 'ibo', split=split) 
  merged_dataset.append(concatenate_datasets([ibo1_split, ibo2_split]))
merged_dataset = DatasetDict({'train':merged_dataset[0], 'validation':merged_dataset[1], 'test':merged_dataset[2]})

In [35]:
merged_dataset

[]

In [9]:
# displays the first element of the training set
merged_dataset["train"][0]["tokens"]

['Ike',
 'ịda',
 'jụụ',
 'otụ',
 'nkeji',
 'banyere',
 'oke',
 'ogbugbu',
 'na',
 '-',
 'eme',
 "n'ala",
 'Naijiria',
 'agwụla',
 'Ekweremmadụ']

In [10]:
#displays the features attribute of our dataset
ner_feature = merged_dataset["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None), length=-1, id=None)

In [11]:
#we can access the list of names in the ner_feature by looking at the names attribute of that feature
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [12]:
#decoding the labels we saw earlier and prints line 1 and line 2 of the training set with the labels
words = merged_dataset["train"][0]["tokens"]
labels = merged_dataset["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Ike ịda jụụ otụ    nkeji  banyere oke ogbugbu na - eme n'ala Naijiria agwụla Ekweremmadụ 
O   O   O   B-DATE I-DATE O       O   O       O  O O   O     B-LOC    O      B-PER       


In [13]:
#mount the gdrive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
#model path in the gdrive
MODEL_PATH = "/content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0" #replace with the model path you want to work with

**Processing the data**

In [15]:
#we will be using a IgboBERT pretrained model.
#Download and cache the associated tokenizer

#You can replace the model_checkpoint with any other model you prefer from the Hub, 
#or with a local folder in which you’ve saved a pretrained model and a tokenizer
from transformers import AutoTokenizer

model_checkpoint = MODEL_PATH
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

In [16]:
#Tokenizing our pre-tokenized input with our tokenizer
inputs = tokenizer(merged_dataset["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['<s>',
 'ĠIke',
 'Ġá»ĭda',
 'Ġjá»¥á»¥',
 'Ġotá»¥',
 'Ġnkeji',
 'Ġbanyere',
 'Ġoke',
 'Ġogbugbu',
 'Ġna',
 'Ġ-',
 'Ġeme',
 'Ġn',
 "'",
 'ala',
 'ĠNaijiria',
 'Ġagwá»¥la',
 'ĠEkweremmadá»¥',
 '</s>']

In [17]:
# Expanding our label list to match the tokens and assigning a label of -100 to special tokens 
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
# trying out the above line of code on our first sentence to see if -100 was assigned to the special tokens
labels = merged_dataset["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]
[-100, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, -100]


In [19]:
#we tokenize all the inputs and apply align_labels_with_tokens() on all the labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [20]:
#applying all that preprocessing in one go on the other splits of our dataset
tokenized_datasets = merged_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=merged_dataset["train"].column_names,
)

  0%|          | 0/10 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

**Fine-tuning the model with the Trainer API**

In [21]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [22]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    7,    8,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    5,    0,    1, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    5,    0,    1,
            2,    0,    0,    0,    0,    0,    0,    0,    0,    7,    8,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            5,    0,    0,    0,    0, -100]])


To have the Trainer compute a metric every epoch, we will need to define a compute_metrics() function that takes the arrays of predictions and    labels, and returns a dictionary with the metric names and values.To use this metric, we first need to install the seqeval library.

In [23]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=fb9f39c1eb5e1d29ed28ad6326c973a0d8466f600bf68bd1f3f2f58090de9c8d
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [24]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-24-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [25]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [26]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [27]:
#defining the model we want to finetune
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at /content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0 were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRA

In [28]:
#log in to Hugging Face 
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [29]:
#we define our TrainingArguments
from transformers import TrainingArguments

args = TrainingArguments(
    "IgboBert-finetuned-ner-merged",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=30,
    weight_decay=0.01,
    #push_to_hub=True, (uncomment if you want to upload your results to the Model Hub)
)

In [31]:
#we pass everything to the Trainer and start training
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

***** Running training *****
  Num examples = 9869
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 37020
  Number of trainable parameters = 82867209


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.112200,0.132796,0.652566,0.753846,0.699559,0.963320
2,0.057500,0.117535,0.683466,0.776244,0.726907,0.971284
3,0.038400,0.123907,0.708495,0.783032,0.743901,0.971315
4,0.023600,0.161321,0.725806,0.763575,0.744212,0.970561
5,0.018300,0.147942,0.741368,0.782127,0.761202,0.972730
6,0.013300,0.194436,0.746066,0.772398,0.759004,0.972162
7,0.010500,0.158652,0.751389,0.795249,0.772697,0.972390
8,0.009300,0.202325,0.746549,0.783032,0.764355,0.972152
9,0.007400,0.195711,0.785376,0.787330,0.786352,0.973980
10,0.006300,0.199497,0.766601,0.796606,0.781316,0.973918


***** Running Evaluation *****
  Num examples = 2819
  Batch size = 8
Saving model checkpoint to IgboBert-finetuned-ner-merged/checkpoint-1234
Configuration saved in IgboBert-finetuned-ner-merged/checkpoint-1234/config.json
Model weights saved in IgboBert-finetuned-ner-merged/checkpoint-1234/pytorch_model.bin
tokenizer config file saved in IgboBert-finetuned-ner-merged/checkpoint-1234/tokenizer_config.json
Special tokens file saved in IgboBert-finetuned-ner-merged/checkpoint-1234/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2819
  Batch size = 8
Saving model checkpoint to IgboBert-finetuned-ner-merged/checkpoint-2468
Configuration saved in IgboBert-finetuned-ner-merged/checkpoint-2468/config.json
Model weights saved in IgboBert-finetuned-ner-merged/checkpoint-2468/pytorch_model.bin
tokenizer config file saved in IgboBert-finetuned-ner-merged/checkpoint-2468/tokenizer_config.json
Special tokens file saved in IgboBert-finetuned-ner-merged/checkpoint-2468/specia

TrainOutput(global_step=37020, training_loss=0.011139742167278692, metrics={'train_runtime': 3113.9708, 'train_samples_per_second': 95.078, 'train_steps_per_second': 11.888, 'total_flos': 4688958682964430.0, 'train_loss': 0.011139742167278692, 'epoch': 30.0})

**Using the fine-tuned model**

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/IgboBert-finetuned-ner/checkpoint-5600"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Google chetara ụbọchị ọmụmụ Keshi mgbe Obasanjo chịrị Naịjirịa afọ asatọ na ọchịchị onye kwuo uche ya."
                 "Onyeisi ndị na-emenyu ọkụ na, Legọọsi steeti bụ Rasak Fadipe ekwuola na onweghi onye nwụrụ n'ime ọkụ ahụ gbara n'ehihie ụbọchị 24 Jenuwarị, 2018.")

loading configuration file /content/IgboBert-finetuned-ner/checkpoint-5600/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/IgboBert-finetuned-ner/checkpoint-5600",
  "architectures": [
    "RobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "p

[{'end': 6,
  'entity_group': 'ORG',
  'score': 0.99998724,
  'start': 0,
  'word': ' Google'},
 {'end': 21,
  'entity_group': 'DATE',
  'score': 0.99948347,
  'start': 15,
  'word': ' ụbọchị'},
 {'end': 33,
  'entity_group': 'PER',
  'score': 0.99567986,
  'start': 28,
  'word': ' Keshi'},
 {'end': 47,
  'entity_group': 'PER',
  'score': 0.99999,
  'start': 39,
  'word': ' Obasanjo'},
 {'end': 62,
  'entity_group': 'LOC',
  'score': 0.9999815,
  'start': 54,
  'word': ' Naịjirịa'},
 {'end': 72,
  'entity_group': 'DATE',
  'score': 0.9996765,
  'start': 63,
  'word': ' afọ asatọ'},
 {'end': 146,
  'entity_group': 'LOC',
  'score': 0.99975836,
  'start': 132,
  'word': ' Legọọsi steeti'},
 {'end': 162,
  'entity_group': 'PER',
  'score': 0.99998444,
  'start': 150,
  'word': ' Rasak Fadipe'},
 {'end': 241,
  'entity_group': 'DATE',
  'score': 0.98287994,
  'start': 214,
  'word': "'ehihie ụbọchị 24 Jenuwarị,"},
 {'end': 246,
  'entity_group': 'DATE',
  'score': 0.99987113,
  'start': 24

In [32]:
#save the fine-tuned model in gdrive
import shutil
shutil.move('/content/IgboBert-finetuned-ner-merged','/content/gdrive/MyDrive/IboBERT_2.0/FINE TUNING/mearged')

'/content/gdrive/MyDrive/IboBERT_2.0/FINE TUNING/mearged/IgboBert-finetuned-ner-merged'

# **END OF CODE**

In [13]:
from datasets import concatenate_datasets, load_dataset 
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='test') 
ibo1 = load_dataset('masakhaner', 'ibo', split='test')
test = concatenate_datasets([ibo2, ibo1])



In [16]:
test

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 2819
})

In [ ]:
from datasets import concatenate_datasets, load_dataset
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='validation')
ibo1 = load_dataset('masakhaner', 'ibo', split='validation')
validation = concatenate_datasets([ibo2, ibo1])

In [ ]:
from datasets import concatenate_datasets, load_dataset
ibo1 = load_dataset('masakhaner', 'ibo', split='train')
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='train')

In [ ]:
# from datasets import load_dataset

# url = "https://github.com/Chiamakac/TRAININGS/tree/main/IgboBERT%202.0/ibo%20merged/"
# data_files = {"dev": f"{url}dev.txt", "test": f"{url}test.txt", "train": f"{url}train.txt"}
# raw_datasets = load_dataset("text", data_files=data_files)

In [ ]:
from datasets import load_dataset

url = "https://github.com/Chiamakac/TRAININGS/tree/main/IgboBERT%202.0/ibo%20merged/"
data_files = {
    "train": url + "train.txt",
    "test": url + "test.txt",
    "dev": url + "dev.txt",
}
raw_datasets = load_dataset("text", data_files=data_files)